<a href="https://colab.research.google.com/github/mahimscit/DO180-apps/blob/master/SQL3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL Mandatory Assignment 3

In [31]:
import sqlite3

def create_and_populate_database(db_file):

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:

        cursor.executescript("""
            CREATE TABLE Restaurants (
                RestaurantID INT PRIMARY KEY,
                RestaurantName TEXT,
                Cuisine TEXT,
                Area TEXT,
                Rating FLOAT,
                AvgCostForTwo INT
            );

            CREATE TABLE Users (
                UserID INT PRIMARY KEY,
                UserName TEXT,
                Email TEXT
            );

            CREATE TABLE Orders (
                OrderID INT PRIMARY KEY,
                UserID INT,
                RestaurantID INT,
                OrderDate DATETIME,
                TotalAmount FLOAT,
                FOREIGN KEY (UserID) REFERENCES Users(UserID),
                FOREIGN KEY (RestaurantID) REFERENCES Restaurants(RestaurantID)
            );

            CREATE TABLE Reviews (
                ReviewID INT PRIMARY KEY,
                UserID INT,
                RestaurantID INT,
                Rating INT,
                ReviewText TEXT,
                FOREIGN KEY (UserID) REFERENCES Users(UserID),
                FOREIGN KEY (RestaurantID) REFERENCES Restaurants(RestaurantID)
            );
        """)


        cursor.executemany("INSERT INTO Restaurants VALUES (?, ?, ?, ?, ?, ?)", [
            (1, 'The Italian Place', 'Italian', 'Downtown', 4.5, 60),
            (2, 'Spice Delight', 'Indian', 'Uptown', 4.2, 50),
            (3, 'Taco Fiesta', 'Mexican', 'Midtown', 4.0, 45),
            (4, 'Sushi Heaven', 'Japanese', 'Downtown', 4.7, 75),
            (5, 'Burger Joint', 'American', 'Uptown', 4.3, 55)
        ])


        cursor.executemany("INSERT INTO Users VALUES (?, ?, ?)", [
            (1, 'Alice', 'alice@example.com'),
            (2, 'Bob', 'bob@example.com'),
            (3, 'Charlie', 'charlie@example.com')
        ])


        cursor.executemany("INSERT INTO Orders VALUES (?, ?, ?, ?, ?)", [
            (1, 1, 2, '2023-11-15 19:30:00', 55.00),
            (2, 2, 1, '2023-11-16 20:15:00', 62.50),
            (3, 3, 4, '2023-11-17 18:45:00', 80.00),
            (4, 1, 3, '2023-11-18 12:30:00', 48.75),
            (5, 2, 5, '2023-11-19 21:00:00', 60.00)
        ])


        cursor.executemany("INSERT INTO Reviews VALUES (?, ?, ?, ?, ?)", [
            (1, 1, 2, 5, 'Amazing food and service!'),
            (2, 2, 1, 4, 'Good Italian food, a bit pricey.'),
            (3, 3, 4, 5, 'Best sushi I''ve ever had!'),
            (4, 1, 3, 3, 'Tacos were okay, but service was slow.'),
            (5, 2, 5, 4, 'Great burgers and fries!')
        ])

        conn.commit()
        print("Database created and populated successfully!")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()


database_file = 'Jomato.db'

1 . Create a stored procedure to display the restaurant name, type and cuisine where the
table booking is not zero.

In [33]:
import sqlite3


conn = sqlite3.connect('Jomato.db')
cursor = conn.cursor()

cursor.execute("ALTER TABLE Restaurants ADD COLUMN TableBooking BOOLEAN;")


cursor.execute("UPDATE Restaurants SET TableBooking = 1 WHERE RestaurantID IN (1, 2, 4);")
cursor.execute("UPDATE Restaurants SET TableBooking = 0 WHERE RestaurantID IN (3, 5);")

conn.commit()


In [38]:
import sqlite3

def alter_and_update_restaurants(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:
        cursor.execute("""
            ALTER TABLE restaurants
            ADD COLUMN restaurant_type TEXT;
        """)

        cursor.execute("""
            UPDATE restaurants SET restaurant_type = 'Casual Dining' WHERE cuisine = 'Italian';
        """)

        cursor.execute("""
            UPDATE restaurants SET restaurant_type = 'Fast Food' WHERE cuisine = 'Burger';
        """)

        cursor.execute("""
            UPDATE restaurants SET restaurant_type = 'Casual Dining' WHERE cuisine = 'Mexican';
        """)

        cursor.execute("""
            UPDATE restaurants SET restaurant_type = 'Casual Dining' WHERE cuisine = 'Indian';
        """)

        conn.commit()

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()

database_file = 'Jomato.db'
alter_and_update_restaurants(database_file)

In [42]:
import sqlite3

def display_restaurants_with_table_booking(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:
        cursor.execute("""
            SELECT r.RestaurantName, r.cuisine, r.restaurant_type
            FROM restaurants r
            JOIN bookings b ON r.RestaurantID = b.restaurant_id
            WHERE b.num_guests > 0
            GROUP BY r.RestaurantID, r.RestaurantName, r.cuisine, r.restaurant_type;
        """)
        results = cursor.fetchall()

        for row in results:
            print(f"Restaurant: {row[0]}, Type: {row[2]}, Cuisine: {row[1]}")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()

database_file = 'Jomato.db'
display_restaurants_with_table_booking(database_file)

2. Create a transaction and update the cuisine type ‘Cafe’ to ‘Cafeteria’. Check the result and rollback it.

In [43]:
import sqlite3

def update_and_rollback_cuisine(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:
        cursor.execute("BEGIN TRANSACTION")

        cursor.execute("UPDATE restaurants SET cuisine = 'Cafeteria' WHERE cuisine = 'Cafe'")

        cursor.execute("SELECT * FROM restaurants WHERE cuisine = 'Cafeteria'")
        updated_rows = cursor.fetchall()

        conn.rollback()

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()

database_file = 'Jomato.db'
update_and_rollback_cuisine(database_file)

In [46]:
conn = sqlite3.connect('Jomato.db')

cursor = conn.execute(''' SELECT * FROM Restaurants; ''')

for row in cursor:
  print(row)

3. Generate a row number column and find the top 5 areas with the highest rating of restaurants.

In [50]:
import sqlite3

def insert_high_rated_restaurants(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:

        cursor.executemany("INSERT INTO Restaurants (RestaurantID, RestaurantName, Cuisine, Area, Rating, AvgCostForTwo, TableBooking, restaurant_type) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", [
            (6, 'The French Bistro', 'French', 'Suburbia', 4.9, 80, 1, 'Fine Dining'),
            (7, 'The Steakhouse', 'Steak', 'Suburbia', 4.8, 90, 1, 'Fine Dining'),
        ])
        conn.commit()
        print("High-rated restaurants inserted successfully!")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()

database_file = 'Jomato.db'
insert_high_rated_restaurants(database_file)

top_5_areas_by_rating(database_file)

High-rated restaurants inserted successfully!
Top 5 Areas with Highest Restaurant Ratings:
Area: Suburbia, Average Rating: 4.85


In [51]:
import sqlite3

def top_5_areas_by_rating(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:
        cursor.execute("""
            SELECT Area, AVG(Rating) as AvgRating,
                   ROW_NUMBER() OVER (ORDER BY AVG(Rating) DESC) as RowNum
            FROM Restaurants
            GROUP BY Area
            ORDER BY AvgRating DESC
            LIMIT 5;
        """)
        results = cursor.fetchall()

        print("Top 5 Areas with Highest Restaurant Ratings:")
        for row in results:
            print(f"Area: {row[0]}, Average Rating: {row[1]:.2f}")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()

database_file = 'Jomato.db'
top_5_areas_by_rating(database_file)

Top 5 Areas with Highest Restaurant Ratings:
Area: Suburbia, Average Rating: 4.85


4. Use the while loop to display the 1 to 50

In [52]:
count = 1

while count <= 50:
    print(count)
    count += 1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


5. Write a query to Create a T op rating view to store the generated top 5 highest rating of restaurants

In [53]:
import sqlite3

def create_top_rating_view(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:
        cursor.execute("""
            CREATE VIEW Top_Rating_Restaurants AS
            SELECT RestaurantName, Rating
            FROM Restaurants
            ORDER BY Rating DESC
            LIMIT 5;
        """)
        conn.commit()
        print("Top_Rating_Restaurants view created successfully!")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()

database_file = 'Jomato.db'
create_top_rating_view(database_file)

Top_Rating_Restaurants view created successfully!


6. Write a trigger that sends an email notification to the restaurant owner whenever a new record is inserted

In [58]:
import sqlite3
import smtplib
from email.mime.text import MIMEText

def send_email_notification():
       sender_email = "mahi@gmail.com"
       sender_password = "abcXXX"
       receiver_email = "ashok@gmail.com"

       msg = MIMEText("A new order has been placed!")
       msg["Subject"] = "New Order Notification"
       msg["From"] = sender_email
       msg["To"] = receiver_email

       with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
           server.login(sender_email, sender_password)
           server.sendmail(sender_email, receiver_email, msg.as_string())

conn = sqlite3.connect("Jomato.db")
conn.create_function("send_email_notification", 0, send_email_notification)
conn.close()